In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# **Cleaning Data Conclusions**

## Day 2
- `final_demo_df`
- `final_experiment_clients_df`

# Final Demo DF

In [ ]:
final_demo_df = pd.read_csv("../../data/clean/df_final_demo_clean.csv")
final_demo_df.head()